# Notebook 07 - Store-Level Concept Matching

This notebook matches recipe ingredients to store-specific product availability using ontology-aligned food concepts.

Key steps:
- Load products with delivery data and ontology concepts
- Load recipes with normalized ingredient concepts
- Group store availability by `product_concept`
- Match ingredients to available store products using `ingredient_concept`

Inputs:
- `products_with_ontology.csv`
- `recipes_with_ontology.csv`

Output:
- `store_recipe_matches.csv` - recipe-to-product mappings per store (by concept)


In [6]:
import pandas as pd
import os

input_folder = "cleaned_data"

# Load enriched datasets
df_products = pd.read_csv(os.path.join(input_folder, "products_with_ontology.csv"))
df_recipes = pd.read_csv(os.path.join(input_folder, "recipes_with_ontology.csv"))

print("Products:", df_products.shape)
print("Recipes:", df_recipes.shape)


C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,3,5,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Products: (126919, 32)
Recipes: (6, 6)


In [7]:
# Keep only rows with known ontology mapping and nonzero delivery
df_filtered = df_products[
    df_products["product_concept"].notna() &
    df_products["delivered_quantity"].fillna(0).astype(float) > 0
].copy()

# Drop any missing delivery info or store IDs
df_filtered = df_filtered.dropna(subset=["store", "article", "product_concept"])

print("Filtered products:", df_filtered.shape)
display(df_filtered[["store", "product_concept", "delivered_quantity"]].head())


Filtered products: (10, 32)


,store,product_concept,delivered_quantity
8058,1024,yogurt,12.0
22210,1058,yogurt,18.0
66841,1160,yogurt,6.0
112267,3298,yogurt,6.0
112268,3298,yogurt,6.0


In [8]:
# Group available concepts per store
store_concept_lookup = (
    df_filtered.groupby("store")["product_concept"]
    .apply(lambda x: set(x.dropna().unique()))
    .to_dict()
)

print(f"Total stores with concept-level availability: {len(store_concept_lookup)}")


Total stores with concept-level availability: 9


In [9]:
results = []

for _, row in df_recipes.iterrows():
    ingr = row["ingredient"]
    ingr_concept = row["ingredient_concept"]

    if not ingr_concept:
        continue  # skip if no ontology match

    for store, concepts in store_concept_lookup.items():
        if ingr_concept in concepts:
            results.append({
                "store": store,
                "ingredient": ingr,
                "ingredient_concept": ingr_concept,
                "match_available": True
            })
        else:
            results.append({
                "store": store,
                "ingredient": ingr,
                "ingredient_concept": ingr_concept,
                "match_available": False
            })

# Convert to DataFrame
df_matches = pd.DataFrame(results)
df_matches.head()


,store,ingredient,ingredient_concept,match_available
0,1024,strawberries,strawberries,False
1,1058,strawberries,strawberries,False
2,1160,strawberries,strawberries,False
3,3298,strawberries,strawberries,False
4,3345,strawberries,strawberries,False


In [10]:
output_path = os.path.join(input_folder, "store_recipe_matches.csv")
df_matches.to_csv(output_path, index=False)
print("-> Saved store-level recipe matches to:", output_path)


-> Saved store-level recipe matches to: cleaned_data\store_recipe_matches.csv
